In [1]:
%cd ../..

/Users/zfac230/botanica_project


/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import copy
import numpy as np
import time
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision

from timm.loss import LabelSmoothingCrossEntropy
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from torchvision import datasets, models
from torchvision import transforms as T

/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


trainset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

testset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [5]:
classes = trainset.classes
num_class = len(classes)

In [6]:
class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
model1 = TinyModel()
model2 = TinyModel()
model3 = TinyModel()
models = [model1, model2, model3]
weights = [0.1, 0.5, 0.8]
threshold=0.5
epochs=3

In [8]:
pso_pop = 7
fa_pop = 3
dim = 3
w = 0.5
alpha_0 = 0.9
alpha_f = 0.1
df = 0.5
states = np.array([[1.3,0.8],
                   [1.2,1.2],
                   [0.6,1.5]])
maxiter = 10
fa_options = {'bmin':1, 'gamma':0.8, 'alpha':0.2}
search_bounds = (np.array([0] * dim), np.array([1] * dim))
k_vals = 0.8, 0.2

In [9]:
model1 = TinyModel().to(mps_device)
model2 = TinyModel().to(mps_device)
model3 = TinyModel().to(mps_device)
models = [model1, model2, model3]
weights = [0.1, 0.5, 0.8]
threshold=0.5
epochs=3

In [10]:
from botanica.transformer.model_selection import _get_model_output
output = _get_model_output(model1, trainloader, mps_device)
len(output)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:02<00:00, 425.36it/s]


60000

In [11]:
from botanica.transformer.model_selection import get_suite_outputs
suite_train_output = get_suite_outputs(models, testloader, mps_device)
len(suite_train_output)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 460.67it/s]


3

In [12]:
suite_val_output = get_suite_outputs(models, testloader, mps_device)
len(suite_val_output)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 398.69it/s]


3

In [13]:
from botanica.transformer.model_selection import get_ensemble_output
ens_train_out = get_ensemble_output(suite_train_output, weights)
ens_val_out = get_ensemble_output(suite_val_output, weights)
print(len(ens_train_out))
print(len(ens_val_out))

10000
10000


In [14]:
sftmx = torch.nn.Softmax(dim=1)

In [15]:
out1 = torch.Tensor([[1,.5,1]])
out2 = torch.Tensor([[1,1,1]])
print(sftmx(out1))
print(sftmx(out2))
weights = [1, 2]
test_out = get_ensemble_output([out1, out2], weights)
test_out

tensor([[0.3837, 0.2327, 0.3837]])
tensor([[0.3333, 0.3333, 0.3333]])


tensor([[1.4340, 1.1321, 1.4340]])

In [16]:
len(models)

3

In [17]:
from botanica.transformer.model_selection import get_model_suite
get_model_suite([0.3, 0.5, 0.8], models, 0.5)

[1, 2]

In [18]:
from botanica.transformer.model_selection import fitness_wrapper_ensemble
num_models = len(suite_val_output)
fitness = fitness_wrapper_ensemble(ens_val_out, num_models, (0.8,0.2), mps_device, classes, testset)

Loss: 2.3026
top_k acc: 0.1010, 0.3000, 0.5003


In [19]:
print(fitness)

tensor(-0.1475, device='mps:0')


Testing tensor loading

In [20]:
torch.save(out1, "model_1_val_logits.pt")
torch.save(out2, "model_2_val_logits.pt")

In [21]:
from botanica.transformer.model_selection import load_output_from_dir

In [22]:
output_tensors = load_output_from_dir('.', 2)

In [23]:
output_tensors

tensor([[[1.0000, 0.5000, 1.0000]],

        [[1.0000, 1.0000, 1.0000]]])

In [24]:
output_tensors[0]

tensor([[1.0000, 0.5000, 1.0000]])

In [25]:
output_tensors[0]

tensor([[1.0000, 0.5000, 1.0000]])

In [26]:
output_tensors.shape

torch.Size([2, 1, 3])

In [27]:
outputs = torch.stack(suite_val_output)
outputs = torch.rand(outputs.shape).to(mps_device)
weights = np.array([1, 1.1, 0.1])
def swarm_wrapper(position):
    val_suite_index = get_model_suite(position, outputs, threshold)
    val_suite = outputs[val_suite_index]
    suite_weights = weights[val_suite_index]
    num_models = len(val_suite)
    val_out = get_ensemble_output(val_suite, weights)
    fitness = fitness_wrapper_ensemble(val_out, num_models, (0.8,0.2), mps_device, classes, testset)
    return fitness

In [28]:
outputs.shape

torch.Size([3, 10000, 10])

In [29]:
swarm_wrapper([1,0,0])

Loss: 2.3037
top_k acc: 0.0995, 0.3038, 0.5016


tensor(-0.2796, device='mps:0')

In [30]:
swarm_wrapper([0,1,0])

Loss: 2.3041
top_k acc: 0.0977, 0.2994, 0.5018


tensor(-0.2782, device='mps:0')

In [31]:
swarm_wrapper([0,0,1])

Loss: 2.3049
top_k acc: 0.0942, 0.2943, 0.4893


tensor(-0.2754, device='mps:0')

In [32]:
outputs[0]

tensor([[0.9602, 0.9199, 0.4060,  ..., 0.8564, 0.9751, 0.5566],
        [0.6508, 0.2272, 0.9987,  ..., 0.6583, 0.0249, 0.6021],
        [0.6796, 0.0462, 0.4574,  ..., 0.6426, 0.1015, 0.8857],
        ...,
        [0.2283, 0.6087, 0.3744,  ..., 0.9334, 0.5637, 0.2899],
        [0.7841, 0.2136, 0.9075,  ..., 0.7599, 0.7701, 0.6572],
        [0.5244, 0.1176, 0.2690,  ..., 0.7935, 0.6329, 0.2126]],
       device='mps:0')

In [33]:
type(testset.targets)

torch.Tensor

In [34]:
torch.stack(suite_val_output).shape

torch.Size([3, 10000, 10])

In [35]:
len(testset)

10000

In [36]:
type(testset.targets) == torch.Tensor

True

In [37]:
type(testset.targets) == list

False

In [38]:
from torch import Tensor

In [39]:
type(testset.targets) == Tensor

True

In [40]:
testset.targets = testset.targets.tolist()

In [42]:
from botanica.transformer.architecture import transformer_ensemble_weighted
ensemble_model = transformer_ensemble_weighted(models, weights)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()